In [1]:
import numpy as np
import py4dgeo
import pickle
from tqdm import tqdm
import os, sys
sys.path.append('../src')
import helper, classes

In [2]:
# _cloudcompare_param_mapping = {
#     "normalscale": "normal_radii",
#     "registrationerror": "reg_error",
#     "searchdepth": "max_distance",
#     "searchscale": "cyl_radii",
#     "usemedian": "robust_aggr",
# }

##### Create class wise point clouds 

In [3]:
def laspy_to_np_array(laspy_points):
    x = laspy_points['x']
    y = laspy_points['y']
    z = laspy_points['z']
    return np.column_stack((x, y, z))

def class_split_pc(points_all_classes, type=None):
    if type == 'real':
        labels = points_all_classes['classification']
        ordered_classes_dict = classes.CLASSES_FOR_M3C2_REAL
    elif type == 'synth':
        labels = points_all_classes.semantic_tags
        ordered_classes_dict = classes.CLASSES_FOR_M3C2_SYNTH
    else:
        return
    
    class_wise_points = []
    for class_number in ordered_classes_dict:
        class_indices = np.isin(labels, class_number)
        class_points = points_all_classes[class_indices]
        class_points_np = laspy_to_np_array(class_points)
        class_wise_points.append(class_points_np)
    
    return class_wise_points

EVERY_NTH = 100

def m3c2_class_wise(real_pc_path, synth_pc_path):
    print('Reading & preparing data')
    real_points_all_classes, synth_points_all_classes = helper.import_and_prepare_point_clouds(real_pc_path, synth_pc_path, shift_real=True, flip_synth=True, crop=True)
    print('Splitting data')
    real_points_class_wise = class_split_pc(real_points_all_classes, type='real')
    synth_points_class_wise = class_split_pc(synth_points_all_classes, type='synth')

    # ensure number of point clouds/classes is the same for real and synth
    assert(len(real_points_class_wise) == len(synth_points_class_wise))
    
    class_wise_distances_all = []
    class_wise_distances_medians = []
    class_wise_uncertainties_all = []
    class_wise_uncertainties_mean = []

    print('Calculating distances...')
    for class_number in tqdm(range(len(real_points_class_wise))):

        # m3c2 needs special epoch data type, timestamp is optional 
        epoch1 = py4dgeo.Epoch(real_points_class_wise[class_number])
        epoch2 = py4dgeo.Epoch(synth_points_class_wise[class_number])
        
        corepoints = epoch1.cloud[::EVERY_NTH]

        #TODO adjust params
        m3c2 = py4dgeo.M3C2(epochs=(epoch1, epoch2),
            corepoints=corepoints,
            cyl_radii=(2.0,),
            normal_radii=(0.5, 1.0, 2.0)
            )
        distances, uncertainties = m3c2.run()

        distances = np.array(distances)
        uncertainties = np.array(uncertainties)

        median_distance = np.median(distances)
        median_uncertainty = np.median(uncertainties)

        class_wise_distances_all.append(distances)
        class_wise_distances_medians.append(median_distance)
        class_wise_uncertainties_all.append(uncertainties)
        class_wise_uncertainties_mean.append(median_uncertainty)

    return class_wise_distances_medians, class_wise_uncertainties_mean, class_wise_uncertainties_all, class_wise_uncertainties_all


In [4]:
synth_pc_path = '/home/Meins/Uni/TUM/SS23/Data Lab/Data Sets/Synthetic/Val_1 - Cloud.las'
real_pc_path = '/home/Meins/Uni/TUM/SS23/Data Lab/Labelling/Label-Datasets/train/Train1 - labelled.las'

In [5]:
#real_points_all_classes, synth_points_all_classes = helper.import_and_prepare_point_clouds(real_pc_path, synth_pc_path, shift_real=True, flip_synth=True, crop=True)
#real_points_class_wise = class_split_pc(real_points_all_classes, type='real')
#synth_points_class_wise = class_split_pc(synth_points_all_classes, type='synth')

In [6]:
distances_medians, uncertainties_mean, uncertainties_all, uncertainties_all = m3c2_class_wise(real_pc_path, synth_pc_path)

Reading & preparing data
Splitting data
Calculating distances...


  0%|          | 0/7 [00:00<?, ?it/s]

<class 'numpy.ndarray'>
[2023-07-11 08:09:19][INFO] Building KDTree structure with leaf parameter 10
[2023-07-11 08:09:22][INFO] Building KDTree structure with leaf parameter 10
